In [78]:
import os
import string

import numpy as np

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Download data

In [27]:
!wget 'http://www.cs.cornell.edu/People/pabo/movie-review-data/rt-polaritydata.tar.gz' -P 'data'
!tar -xf data/rt-polaritydata.tar.gz -C 'data'

--2021-07-29 10:35:56--  http://www.cs.cornell.edu/People/pabo/movie-review-data/rt-polaritydata.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz [following]
--2021-07-29 10:35:56--  http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz
Reusing existing connection to www.cs.cornell.edu:80.
HTTP request sent, awaiting response... 200 OK
Length: 487770 (476K) [application/x-gzip]
Saving to: ‘data/rt-polaritydata.tar.gz’

rt-polaritydata.tar 100%[===================>] 476.34K   512KB/s    in 0.9s    

2021-07-29 10:35:57 (512 KB/s) - ‘data/rt-polaritydata.tar.gz’ saved [487770/487770]



In [64]:
path_ = './data/rt-polaritydata/'
def load_polarity(path=path_):
    data = []
    labels = []
    f_names = ['rt-polarity.neg', 'rt-polarity.pos']
    for (l, f) in enumerate(f_names):
        for line in open(os.path.join(path, f), 'rb'):
            data.append(line.decode('utf8', errors='ignore').strip())
            labels.append(l)
    return data, labels

x_d, y = load_polarity()

x_d[11], y[11]

('while the performances are often engaging , this loose collection of largely improvised numbers would probably have worked better as a one-hour tv documentary .',
 0)

In [66]:
remove_punct = str.maketrans('', '', string.punctuation)
remove_digits = str.maketrans('', '', string.digits)
stops = stopwords.words('english')
wnl = WordNetLemmatizer()
def process_text(sentence):
    out = sentence.lower()
    out = out.translate(remove_punct)
    out = out.translate(remove_digits)
    out = [wnl.lemmatize(w) for w in out.split() if w not in stops]
    return ' '.join(out)

x = []
for x_i in x_d[:]:
    x.append(process_text(x_i))
    
x[11]

'performance often engaging loose collection largely improvised number would probably worked better onehour tv documentary'

In [79]:
x_train_all_d, x_test_d, y_train_all_d, y_test_d = train_test_split(
    x_d, y, test_size=.2, random_state=42)
x_train_d, x_val_d, y_train_d, y_val_d = train_test_split(
    x_train_all_d, y_train_all_d, test_size=.1, random_state=42)

x_train_all, x_test, y_train_all, y_test = train_test_split(
    x, y, test_size=.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(
    x_train_all, y_train_all, test_size=.1, random_state=42)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

In [85]:
class VectorizeText():
    def __init__(self, limit=True, **kwargs):
      if limit:
        self.count_vect = CountVectorizer(min_df=0.003, max_df=0.997, **kwargs)
      else:
        self.count_vect = CountVectorizer(**kwargs)
      self.tf_transformer = TfidfTransformer(use_idf=False)
    def fit(self, x):
        x = self.count_vect.fit_transform(x)
        self.tf_transformer.fit(x)
    def transform(self, x):
        x = self.count_vect.transform(x)
        x = self.tf_transformer.transform(x)
        return x.toarray()     
    def get_feature_names(self):
        return vectorizer.count_vect.get_feature_names()
    
vectorizer = VectorizeText(limit=True) # max_features=500)
#vectorizer = VectorizeText(limit=True, stop_words='english')
vectorizer.fit(x_train)    
x_vec_train = vectorizer.transform(x_train)
x_vec_test = vectorizer.transform(x_test)

names_features = np.array(vectorizer.get_feature_names())
names_features[:10], len(names_features)

(array(['act', 'acted', 'acting', 'action', 'actor', 'actress', 'actually',
        'adaptation', 'add', 'adult'], dtype='<U14'),
 523)